In [0]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score, \
                            precision_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
import random
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc  
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
rand_seed = 123
random.seed(rand_seed)
np.random.seed(rand_seed)


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_rf = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/train_1793_7172_rf.csv')
train_rf = train_rf.drop(['Unnamed: 0','bidprice', 'payprice'], axis = 1)

In [95]:
train_rf.head()

,hour,slotprice,weekday_6,slotvisibility_FirstView,slotformat_5,domain1,domain2,domain3,domain4,domain5,...,keypage3,usertag_10006,usertag_10063,usertag_10031,usertag_11278,usertag_10684,usertag_13042,os_android,os_windows,browser_safari
0,13,0,0,1,0,5.0,1.0,-3.0,4.0,0.0,...,-1.0,1,0,0,0,0,0,0,1,0
1,1,0,0,0,0,2.0,-4.0,1.0,1.0,0.0,...,2.0,0,1,0,0,0,0,0,1,0
2,23,0,0,1,0,5.0,-2.0,-5.0,-1.0,0.0,...,-1.0,0,0,0,0,0,0,0,1,0
3,22,0,0,0,0,1.0,-2.0,2.0,0.0,0.0,...,0.0,0,1,0,0,0,0,0,1,0
4,12,20,0,0,0,1.0,-2.0,1.0,0.0,1.0,...,2.0,0,0,0,0,0,0,0,1,0


In [0]:
val_rf = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/val_1793_7172_rf.csv')
val_rf = val_rf.drop(['Unnamed: 0','bidprice','payprice'], axis = 1)

In [97]:
val_rf.head()

,hour,slotprice,weekday_6,slotvisibility_FirstView,slotformat_5,domain1,domain2,domain3,domain4,domain5,...,keypage3,usertag_10006,usertag_10063,usertag_10031,usertag_11278,usertag_10684,usertag_13042,os_android,os_windows,browser_safari
0,20,0,0,0,0,-2.0,-2.0,0.0,-1.0,0.0,...,0.0,0,0,0,0,0,0,0,1,0
1,21,0,0,0,0,0.0,-2.0,1.0,1.0,0.0,...,2.0,1,1,0,0,0,0,0,1,0
2,8,5,0,0,0,0.0,-2.0,0.0,-2.0,0.0,...,1.0,1,1,0,0,0,0,0,1,0
3,15,5,0,0,0,-1.0,1.0,-1.0,2.0,-1.0,...,1.0,1,1,0,0,0,0,0,1,0
4,18,5,0,1,0,-2.0,1.0,3.0,2.0,0.0,...,9.0,1,1,0,0,0,0,0,1,0


In [0]:
# xg_columns = list(train_xg.columns)
rf_columns = list(train_rf.columns)

In [0]:
test = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/FINAL_TEST_v1.csv')

In [0]:
test_rf = test[rf_columns]

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/train_1793_7172.csv')

In [0]:
validation = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/FINAL_VALIDATION_v1.csv')

In [0]:
train_click = train['click']

In [0]:
validation_click = validation['click']

In [0]:
validation_bidid = validation['bidid']

In [0]:
validation_payprice = validation['payprice']

In [0]:
del train
del validation

In [0]:
def classifier(model,  x_train, y_train, x_val, y_val):
        # Fit classifier to training data
    % time model.fit(x_train, y_train)
        # Predict on validation set
    preds = model.predict(x_val)

        # Predicted probabilities
    pred_proba = model.predict_proba(x_val)
    
    pred_proba_click = [p[1] for p in pred_proba]
  
        # Evaluate performance
    print("\n")
    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))  #把0分为0 的个数，0-1，1-0，1-1
    print("\n")
    
        # The most important measure is TP/FP rate for the clicked class
    kpi = precision_score(y_val, preds)
    print("Click Precision: {}".format(kpi))
    
    accuracy = accuracy_score(y_val, preds)
    print("\n")
    print("Prediction accuracy: {}".format(accuracy))
    print("\n")
    
    recall = recall_score(y_val, preds)
    print("Recall score: {}".format(recall))
    print("\n")   
    
    roc_auc = roc_auc_score(y_val, preds)
    print("ROC and AUC Score: {}".format(roc_auc))
    print("\n")   
    
    f1 = f1_score(y_val,preds)
    print("f1 score: {}".format(f1))
    print("\n") 
    
    return  pred_proba_click

In [0]:
def recalibrate(ctr):
  w =  7172 / (2430981-1793) 
  pctr = ctr/(ctr + (1- ctr) / w)
  
  return pctr

In [0]:
def non_linear(ctr, validation_payprice):
#   range_all = np.arange(1, 200, 10)
  range_all = np.arange(100, 500, 10)
  lamda_list = np.arange(0.001, 0.002, 0.0002)
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  for c in range_all:
    for lamda in lamda_list:
      
      current_bid = c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3)))    

      budget = 6250 * 1000
      clicks = 0
      impressions = 0
    
      for j in range(len(current_bid)):
        payprice = validation_payprice[j]
        if current_bid[j] > payprice:
          budget = budget - payprice
          if budget >= 0:
            impressions += 1
            if(validation_click[j] == 1):
              clicks += 1
          else:
            budget = budget + payprice
            break
          
      click_list.append(clicks)
      budget_list.append(budget)
      impression_list.append(impressions)
      bid_price.append(current_bid)
#     print("\n")
      print('C = {},lamda = {}, clicks = {}, impression = {}, budget = {}'.format(c, lamda, clicks, impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
#   c_optimal = range_all[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
#   lamda_optimal = lamda_list[index_click]
  print('Maxium click is {}, budget = {}, impression = {}'.format(max_click, budget_max, impression_max))

  return  bid_price_final

In [0]:
def linear(ctr, validation):
  
#   w = 7172/(2430981-1793)
#   pctr = ctr/(ctr + (1-ctr) / w)
#   avgctr = 1793/2430981
#   avgctr_2 = pctr.mean()
  
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  base_list = []
  
  for base in range(50,80,1):
    current_bid = base * ctr  / ctr.mean()
    budget = 6250*1000
    clicks = 0
    impressions = 0
   
    for j in range(len(current_bid)):
      payprice = validation_payprice[j]
      if current_bid[j] > payprice:
        budget = budget - payprice
        if budget >= 0:
          impressions += 1
          if(validation_click[j] == 1):
            clicks += 1
        else:
          budget = budget + payprice
          break
          
    click_list.append(clicks)
    budget_list.append(budget)
    impression_list.append(impressions)
    bid_price.append(current_bid)
    base_list.append(base)
    print('base = {}, clicks = {}, impression = {}, budget = {}'.format(base, clicks,  impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
  base_optimal = base_list[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  print('Maxium is {}, when base = {}, budget = {}'.format(max_click, base_optimal, budget_max, impression_max))
  
  return bid_price_final

# Tune parameter

In [0]:

rf_grid = {
           "n_estimators": [120, 300, 500, 800, 1000, 1200], #
           "max_depth": [5,8,15] #
          }


In [0]:
def tune_parameter(model, grid, x_train, y_train, x_val, y_val):
   
    grid = GridSearchCV(model, 
                        param_grid = grid, 
                        scoring = "f1", 
                        cv = StratifiedKFold(n_splits = 3, shuffle = True, random_state = rand_seed)
                        )
    # Fit grid on train data
    grid.fit(x_train, y_train)
    
#     # Store best model
    best_model = grid.best_estimator_
    print("\n")
    print("Best parameters: {}".format(grid.best_params_))
    
    # Predict on validation set
    preds = grid.predict(x_val)
    
#     # Predicted probabilities
#     pred_proba = grid.predict_proba(x_val)
#     pred_proba_click = [p[1] for p in pred_proba]
    
    # Evaluate performance
    print("\n")
    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))  #把0分为0 的个数，0-1，1-0，1-1
    print("\n")
    
    kpi = precision_score(y_val, preds)
    print("Click Precision: {}".format(kpi))
    
    accuracy = accuracy_score(y_val, preds)
    print("\n")
    print("Prediction accuracy: {}".format(accuracy))
    print("\n")
    
    recall = recall_score(y_val, preds)
    print("Recall score: {}".format(recall))
    print("\n")   
    
    roc_auc = roc_auc_score(y_val, preds)
    print("ROC and AUC Score: {}".format(roc_auc))
    print("\n")   
   

# XGboost

In [0]:
#Tune parameter
tune_parameter(XGBClassifier(random_state = rand_seed), xg_grid, train_xg, train_click, val_xg, validation_click)

In [0]:
ctr_xg = classifier(XGBClassifier(n_estimators = 500, max_depth = 8, learning_rate = 0.001, random_state = rand_seed), 
                    train_xg,train_click, val_xg, validation_click)

In [0]:
pctr_xg = recalibrate(ctr_xg)

In [0]:
bidprice_val_xg = non_linear(pctr_xg, val_payprice)

# Random Forest

In [0]:
tune_parameter(RandomForestClassifier(random_state = rand_seed), rf_grid, train_rf, train_click, val_rf, validation_click)

In [0]:
'''
Best parameters: {'max_depth': 15, 'n_estimators': 1000}


Confusion Matrix:
[[296756   6967]
 [    95    107]]


Click Precision: 0.0151258128357365


Prediction accuracy: 0.9767640042773711


Recall score: 0.5297029702970297


ROC and AUC Score: 0.7533821528951128

800比较好
'''

In [113]:
ctr_rf = classifier(RandomForestClassifier( n_estimators = 1000 ,random_state = rand_seed, max_depth= 15), 
                    train_rf,train_click, val_rf, validation_click)

CPU times: user 11.9 s, sys: 6.06 ms, total: 12 s
Wall time: 12 s


Confusion Matrix:
[[296756   6967]
 [    95    107]]


Click Precision: 0.0151258128357365


Prediction accuracy: 0.9767640042773711


Recall score: 0.5297029702970297


ROC and AUC Score: 0.7533821528951128


f1 score: 0.029411764705882353




In [114]:
pctr_rf = recalibrate(np.array(ctr_rf))
pctr_rf

array([0.00016424, 0.00026746, 0.00022965, ..., 0.00041847, 0.00033813,
       0.00032033])

In [0]:
bidprice_val_rf = non_linear(np.array(ctr_rf), validation_payprice)

In [0]:
biprice_val_rf_linear = linear(np.array(ctr_rf), validation_payprice) #base = 74, 154  #数据去除了bidprice payprice

In [0]:
rf_ctr = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/rf_ctr.csv')

In [13]:
rf_ctr.columns

Index(['Unnamed: 0', 'bidid', 'click_proba'], dtype='object')

In [0]:
rf_ctr = rf_ctr.drop('Unnamed: 0', axis = 1)

In [0]:
biprice_val_rf_linear = linear(rf_ctr['click_proba'], validation_payprice) #数据没有去bidprice payprice base = 74, 161